#Gauss-Seidel Method and L1smoother Kira Novitchkova-Burbank

## Stationary iterative method

A stationary iterative method for solving $Ax = b$ can be defined on the basis of a matrix $B$ which is much simpler to solve system with than with $A$.For exmaple,$B$ can be a diagonal matrix, or an invertible(lower or upper) triangular matrix. Then a simple stationary iterative method reads as follow:

Given an invertible matrix $B$, for solving $Ax=b$, we proceed as follows: \\
$\bullet$ Choose an arbitrary $x_0$(zero or random); \\
$\bullet$ For $k$ = 1,2,...,until a stopping criterion is satisfied, we solve for $x_k$ the system for equations with $B$:

#<center>$B(x_k - x_{k-1}) = b - Ax_{k_1}$</center>
Equivalently, we compute the residual
#<center> $r_{k-1} = b - Ax_{k-1}$ </center>
and then solve for the correction $c_k$
#<center> $Bc_k = r_{k-1}$ </center>
The next iterate $x_k$ is obtained by updating the previous one, $x_{k-1}$ with the correction $c_k$,
#<center> $x_k = x_{k-1} + c_k$ </center>


# $L_1 smoother$
Let $D$ = diag($d_i$) where $d_i$ = $\sum_{j=1}^{n} |a_{ij}|$, the rowsum of the absolute values of the entries of $A$ in each row, $D$ is referred to as the $l_1$ smoother. It has the property that $D - A$ is symmetric positive semidefinite,hence it provides a convergent stationary iteration method.

## $Gauss-Seidel$ method

Let $A$ be split as $A = D + L + L^{T}$, where $D$ is its diagonal and $L$ is its strictly lower triangular part.

$\bullet$ Consider $B = D + L$ which is an invertible lower triangular matrix. It gives a stationary iterative method referred to as the (forward) Gauss-Seidel method.

$\bullet$ Similarly, if $B = D + L^{T}$(the upper triangular part of A) gives a stationary iterative method referred to as the (backward)Gauss-Seidel method

$\bullet$ Finally, the factored matrix $B = (D+L)D^{-1}(D+L^{T})$ gives a convergent stationary iterative method referred to as the symmetric Gauss-Seidel iteration method.

# Your tasks
Write an iterative algorithm for solving $Ax = b$ for a given sparse s.p.d matrix $A$ stored in $csr$ format. The ```function``` should take as input: \\
$\bullet$ matrix ```A```, ```b```,initial iterate ```x_0```, maximal number of iterations ```max_iter```, tolerance ```e```. \\
$\bullet$ on ouput produces: number of iterations, iter, residual norm δ = $||r||$;accuracy achieved, i.e, $\frac{||r||}{||r_0||}$, where δ = $||r_0||$ is the norm of the initial residual.

For iteration matrix B, use: \\
(i) the diagonal matrix $D$ representing the $l_1$ - smoother. \\
(ii) the forward Gauss-Seidel iteration matrix preferably implemented as lower triangular solve (from Homework 1). \\
(iii) the backward Gauss-Seidel iteration matrix preferably as  upper triangular solve (from Homework 1). \\
(iv) the symmetric Gauss-Seidel iteration matrix preferably as two consecutive calls (one to lower triangular solve and another one to upper triangular solve). \\

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from pandas import read_csv

In [ ]:
def read_matrix(filepath):
    df = read_csv(filepath,
                  sep=" ",
                  header=None)
    df.columns = ["col", "row", "data"]
    mtx = coo_matrix((df["data"], (df["row"], df["col"]))).tocsr()
    return mtx


In [ ]:
matrix_25 = read_matrix('25-1.txt') #was read data

# make inverse diagonal matrix
Here we need to write a function that takes ```A``` in $csr$ format and return it's inverse diagal.

# Forward and backward substitution function
Use the function you wrote from homework 1 or use a package.

In [ ]:
from scipy.sparse.linalg import splu

def LU_factor(sparse_matrix):

  lu = splu(matrix_25.tocsc(),
          permc_spec="NATURAL",
          diag_pivot_thresh=0,
          options={"SymmetricMode":True})
  return lu.L , lu.U


####Making lower_matrix and upper_matrix

In [ ]:
#Code from previous HW
lower_matrix,upper_matrix = LU_factor(matrix_25)

#Ly=b
converted_to_csr_lower = lower_matrix.tocsr()

#Ux=y
converted_to_csr_upper = upper_matrix.tocsr()


####Making forward_subs, backward_sub, and L1_Smoother

In [ ]:
#Some new code and some code from previous HW
#Ly=b
def forward_subs(lower, b):

  n = matrix_25.shape[0]
  x = np.ones(n)
  y = np.zeros(n)

  for i in range(0, n):

    y[i] = b[i]

    for k in range(converted_to_csr_lower.indptr[i], converted_to_csr_lower.indptr[i+1]):
      j = converted_to_csr_lower.indices[k]

      if (j < i):
        y[i] -= converted_to_csr_lower.data[k] * y[j]

      if (j == i):
        diag = converted_to_csr_lower.data[k]

    y[i] /= diag

  return y


#Ux=y
def backward_sub(upper, y):

  n = matrix_25.shape[0]
  x = np.zeros(n)

  for i in range(n-1, -1, -1):

    x[i] = y[i]

    for k in range(converted_to_csr_upper.indptr[i+1], converted_to_csr_upper.indptr[i], -1):
      j = converted_to_csr_upper.indices[k-1]

      if (j > i):
        x[i] -= converted_to_csr_upper.data[k-1] * x[j]

      if (j == i):
        diag = converted_to_csr_upper.data[k-1]

    x[i] /= diag

  return x


#Note: this is L1_Smoother but it computes the inverse (1/sum) inside of it, as well.
def L1_Smoother(A):

    n = matrix_25.shape[0]
    sum = 0
    Diag_Inv = np.zeros(n)

    for i in range(0, n):
      for k in range(converted_to_csr.indptr[i], converted_to_csr.indptr[i+1]):
        j = converted_to_csr.indices[k]

        if (i <= j):
          sum += abs(converted_to_csr.data[k])

      Diag_Inv[i] = 1/sum

    return Diag_Inv


#Make_D makes a regular Diagonal Matrix
def Make_D(A):

    diagonal_mat = np.zeros((matrix_25.shape[0], matrix_25.shape[1]))

    for i in range(0, matrix_25.shape[0]):
      for k in range(converted_to_csr_for_Make_D.indptr[i], converted_to_csr_for_Make_D.indptr[i+1]):
        j = converted_to_csr_for_Make_D.indices[k]

        if (i != j):
          converted_to_csr_for_Make_D.data[k] = 0

    diagonal_mat = converted_to_csr_for_Make_D

    return diagonal_mat

#main
#These will be used below
b = np.ones(matrix_25.shape[0])

converted_to_csr = matrix_25.tocsr()

converted_to_csr_for_Make_D = converted_to_csr.copy() #otherwise will change a lot of values to 0's in original matrix

Diagonal_Inverse = L1_Smoother(converted_to_csr)
D = Make_D(converted_to_csr_for_Make_D)


####Diagonal, Forward_GS, Backward_GS, and Symmetric_GS



In [ ]:
from numpy import linalg


def Diagonal(A, b, x0, e, max_iter):
  r = b - A * x0
  delta0 = np.linalg.norm(r)
  x = x0
  delta = delta0
  iter = 0
  while delta > e*delta0 and iter < max_iter:
    c = Diagonal_Inverse @ r
    x = x + c
    r = b - A * x
    delta = np.linalg.norm(r)
    iter += 1

  return x, delta, delta0, iter


def Forward_GS(A, b, x0, e, max_iter):
  r = b - A * x0
  delta0 = np.linalg.norm(r)
  x = x0
  delta = delta0
  iter = 0
  while delta > e*delta0 and iter < max_iter:
    c = forward_subs(A, r)
    x = x + c
    r = b - A * x
    delta = np.linalg.norm(r)
    iter += 1

  return x, delta, delta0, iter


def Backward_GS(A, b, x0, e, max_iter):
  r = b - A * x0
  delta0 = np.linalg.norm(r)
  x = x0
  delta = delta0
  iter = 0
  while delta > e*delta0 and iter < max_iter:
    c = backward_sub(A, r)
    x = x + c
    r = b - A * x
    delta = np.linalg.norm(r)
    iter += 1

  return x, delta, delta0, iter


def Symmetric_GS(A, b, x0, e, max_iter):
  r = b - A * x0
  delta0 = np.linalg.norm(r)
  x = x0
  delta = delta0
  iter = 0
  while delta > e*delta0 and iter < max_iter:
    y = forward_subs(A, r)
    c = backward_sub(A, D*y)

    x = x + c
    r = b - A * x
    delta = np.linalg.norm(r)
    iter += 1

  return x, delta, delta0, iter



####Calling and Testing the Above Functions
#####Note: the best approximations are those that have delta/delta0 < e

In [ ]:
A = matrix_25
x0 = np.zeros(matrix_25.shape[0])
e = 0.000001  #10^-6 #e is epsilon
max_iter = 1000



#Diagonal
x, delta, delta0, iter = Diagonal(A, b, x0, e, max_iter) #Test shows that delta_div_delta0 is way bigger than e

delta_div_delta0 = delta / delta0
print("delta_div_delta0 Diagonal:", delta_div_delta0)
print("iter Diagonal:", iter)

if delta_div_delta0 < e:
  print("delta/delta0 is less than e for Diagonal")
else:
  print("delta/delta0 is either equal or more than e for Diagonal")



#Forward_GS
x, delta, delta0, iter = Forward_GS(A, b, x0, e, max_iter) #Test shows that delta_div_delta0 is e-06 so it's just slightly more than e

delta_div_delta0 = delta / delta0
print("\ndelta_div_delta0 Forward_GS:", delta_div_delta0)
print("iter Forward_GS:", iter)

if delta_div_delta0 < e:
  print("delta/delta0 is less than e for Forward_GS")
else:
  print("delta/delta0 is either equal or more than e for Forward_GS")



#Backward_GS
x, delta, delta0, iter = Backward_GS(A, b, x0, e, max_iter) #Test shows that delta_div_delta0 is less than e (this is a really good approximation)

delta_div_delta0 = delta / delta0
print("\ndelta_div_delta0 Backward_GS:", delta_div_delta0)
print("iter Backward_GS:", iter)

if delta_div_delta0 < e:
  print("delta/delta0 is less than e for Backward_GS")
else:
  print("delta/delta0 is either equal or more than e for Backward_GS")



#Symmetric_GS
x, delta, delta0, iter = Symmetric_GS(A, b, x0, e, max_iter) #Test shows that delta_div_delta0 is e-06 so it's just slightly more than e

delta_div_delta0 = delta / delta0
print("\ndelta_div_delta0 Symmetric_GS:", delta_div_delta0)
print("iter Symmetric_GS:", iter)

if delta_div_delta0 < e:
  print("delta/delta0 is less than e for Symmetric_GS")
else:
  print("delta/delta0 is either equal or more than e for Symmetric_GS")


delta_div_delta0 Diagonal: 1.0187554408954276e+90
iter Diagonal: 1000
delta/delta0 is either equal or more than e for Diagonal

delta_div_delta0 Forward_GS: 6.342344728972894e-06
iter Forward_GS: 1000
delta/delta0 is either equal or more than e for Forward_GS

delta_div_delta0 Backward_GS: 9.551583233804867e-07
iter Backward_GS: 7
delta/delta0 is less than e for Backward_GS

delta_div_delta0 Symmetric_GS: 4.969919757320232e-06
iter Symmetric_GS: 1000
delta/delta0 is either equal or more than e for Symmetric_GS
